In [ ]:
# Constants
MAX_NO_ROWS = 1000000
LOAD_MODEL=True
SKIP_TRAINING=True
FEATURE = 'ft_kettle'
SEQ_PER_BATCH = 1024
SEQ_LEN = 32
N_STRIDES = 1
NORM_TYPE = 'mean'
LEARNING_RATE = 1e-3
CLIP_VALUE = 10.
EPOCHS = 40
ALGO = 'cnn'
CHECKPOINT_PATH = f"{ALGO}_training_{FEATURE}/cp.ckpt"

In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow import keras

%matplotlib inline
print(tf.version.VERSION)

2.11.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Dissertation

/content/drive/My Drive/Colab Notebooks/Dissertation


In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lib.chunk_reader import read_labels, get_merged_chunks, get_dates
from lib.plotting import plot_ft_days, plot_model_history, plot_prediction_windows
from lib.analysis import print_heads_tails

In [ ]:
labels = read_labels(1)
for house in range(1,2):
    print('House {}: '.format(house), labels[house], '\n')

House 1:  ['mains_active', 'mains_apparent', 'mains_rms', 'aggregate_apparent', 'ft_boiler', 'ft_solar_thermal_pump', 'ft_washing_machine', 'ft_dishwasher', 'ft_tv', 'ft_kitchen_lights', 'ft_htpc', 'ft_kettle', 'ft_toaster', 'ft_fridge', 'ft_microwave', 'ft_amp_livingroom', 'ft_adsl_router', 'ft_livingroom_s_lamp', 'ft_lighting_circuit', 'ft_subwoofer_livingroom', 'ft_livingroom_lamp_tv', 'ft_kitchen_phone&stereo', 'ft_coffee_machine', 'ft_gas_oven', 'ft_data_logger_pc', 'ft_office_lamp2', 'aggregate_active', 'unix_time'] 



In [ ]:
ref_chunk_df = get_merged_chunks(1, 1)
print('Original DF shape', ref_chunk_df.shape)
print('---')
ref_chunk_df = ref_chunk_df[0:MAX_NO_ROWS]

reading house 1; chunk 1
reading file data/ukdale-parsed-chunks/house_1/chunk_1.dat; for house 1 and chunk 1
read house 1; chunk 1; df.shape is (4500000, 29)
Original DF shape (4500000, 29)
---


In [ ]:
dates = {}
# for house in range(1,2):
dates[1] = get_dates(ref_chunk_df, 1)

House 1 data contain 71 days from 2013-04-11 to 2013-06-20.
['2013-04-11', '2013-04-12', '2013-04-13', '2013-04-14', '2013-04-15', '2013-04-16', '2013-04-17', '2013-04-18', '2013-04-19', '2013-04-20', '2013-04-21', '2013-04-22', '2013-04-23', '2013-04-24', '2013-04-25', '2013-04-26', '2013-04-27', '2013-04-28', '2013-04-29', '2013-04-30', '2013-05-01', '2013-05-02', '2013-05-03', '2013-05-04', '2013-05-05', '2013-05-06', '2013-05-07', '2013-05-08', '2013-05-09', '2013-05-10', '2013-05-11', '2013-05-12', '2013-05-13', '2013-05-14', '2013-05-15', '2013-05-16', '2013-05-17', '2013-05-18', '2013-05-19', '2013-05-20', '2013-05-21', '2013-05-22', '2013-05-23', '2013-05-24', '2013-05-25', '2013-05-26', '2013-05-27', '2013-05-28', '2013-05-29', '2013-05-30', '2013-05-31', '2013-06-01', '2013-06-02', '2013-06-03', '2013-06-04', '2013-06-05', '2013-06-06', '2013-06-07', '2013-06-08', '2013-06-09', '2013-06-10', '2013-06-11', '2013-06-12', '2013-06-13', '2013-06-14', '2013-06-15', '2013-06-16', '

In [ ]:
# Constants 2
rows_per_day = ref_chunk_df.loc[:dates[1][2]].shape[0]

In [ ]:
from algos.processing import split_df_by_dates
from algos.cnn import create_model, train, predict
from algos.norm import get_ref_norm, denormalize, normalize

In [ ]:
# Separate house 1 data into train, validation and test data
df_train, df_val, df_test  =  split_df_by_dates(ref_chunk_df, dates=dates, house=1)

df_train.shape:  (521300, 29)
df_val.shape:  (316800, 29)
df_test.shape:  (190700, 29)


In [ ]:
ref_norm = get_ref_norm(df_train, NORM_TYPE)

ref_mean: 325.09124755859375; ref_max: 6002.919921875; ref_std: 434.2580871582031
norm_type: mean; ref_norm 325.09124755859375


In [ ]:
model = create_model(seq_len=SEQ_LEN, learning_rate=LEARNING_RATE, clipvalue=CLIP_VALUE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 32, 30)            330       
                                                                 
 conv1d_1 (Conv1D)           (None, 32, 30)            7230      
                                                                 
 conv1d_2 (Conv1D)           (None, 32, 40)            7240      
                                                                 
 conv1d_3 (Conv1D)           (None, 32, 50)            10050     
                                                                 
 conv1d_4 (Conv1D)           (None, 32, 50)            12550     
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 1024)              1

In [ ]:
# !!!!!!!Counter!!!!!!! 
# Manually update for every new loaded training
# Run Counter = 0

# Load and or Train
if LOAD_MODEL:
  model.load_weights(CHECKPOINT_PATH)
if SKIP_TRAINING:
  pass
else:
  model, history, time_spent = train(model, feature=FEATURE, df_train=df_train, df_val=df_val, ref_norm=ref_norm, seq_len=SEQ_LEN, seq_per_batch=SEQ_PER_BATCH, epochs=EPOCHS, checkpoint_path=CHECKPOINT_PATH)

x_train.shape, y_train.shape, x_val.shape, x_val.shape
(521300, 1) (521300, 1) (316800, 1) (316800, 1)
self.data_len 521300
seq_len 32
n_sequences 521268
x.shape, y.shape (521268, 32, 1) (521268,)
seq_index 0
seq_index 160000
seq_index 320000
seq_index 480000
self.data_len 316800
seq_len 32
n_sequences 316768
x.shape, y.shape (316768, 32, 1) (316768,)
seq_index 0
seq_index 160000
Epoch 1/40
510/510 [==============================] - ETA: 0s - loss: 0.1340 - root_mean_squared_error: 0.3660 - mean_absolute_error: 0.0487
Epoch 1: saving model to seq2seq_training_ft_kettle/cp.ckpt
510/510 [==============================] - 22s 20ms/step - loss: 0.1340 - root_mean_squared_error: 0.3660 - mean_absolute_error: 0.0487 - val_loss: 0.0890 - val_root_mean_squared_error: 0.2983 - val_mean_absolute_error: 0.0301
Epoch 2/40
507/510 [============================>.] - ETA: 0s - loss: 0.0916 - root_mean_squared_error: 0.3026 - mean_absolute_error: 0.0307
Epoch 2: saving model to seq2seq_training_ft_ket

Time spent 353.9855970879999
dict_keys(['loss', 'root_mean_squared_error', 'mean_absolute_error', 'val_loss', 'val_root_mean_squared_error', 'val_mean_absolute_error'])


In [ ]:
if not SKIP_TRAINING:
    print('Time spent', time_spent)
    plot_model_history(history)

In [ ]:
# Predict from test split
y_test, y_pred = predict(model, feature=FEATURE, df_test=df_test, ref_norm=ref_norm, seq_len=SEQ_LEN, seq_per_batch=SEQ_PER_BATCH)

x_test.shape, y_test.shape
(190700, 1) (190700, 1)
self.data_len 190700
seq_len 32
n_sequences 190668
x.shape, y.shape (190668, 32, 1) (190668,)
seq_index 0
seq_index 160000
187/187 [==============================] - 1s 5ms/step
y_test.shape, y_pred.shape
(190668,) (190668, 1)
y_test.shape, y_pred.shape
(190668,) (190668,)
Normalized: y_test.mean, y_pred.mean
0.03982254 0.0402174
Denormalized: y_test.mean, y_pred.mean
12.945961 13.104956


In [ ]:
# plot_prediction_windows(FEATURE, y_test_denorm, y_pred_denorm)

In [ ]:
n_samples = int(rows_per_day * 0.2)
plot_prediction_windows(FEATURE, y_test, y_pred, use_active=False, n_samples=n_samples)

In [ ]:
n_samples = 80
plot_prediction_windows(FEATURE, y_test, y_pred, use_active=True, active_factor=4, n_samples=n_samples)

In [ ]:
n_samples = 200
plot_prediction_windows(FEATURE, y_test, y_pred, use_active=True, n_samples=n_samples)

In [ ]:
from lib import analysis
from lib.logging import losses_to_file, metrics_to_file

In [ ]:
params = {}
params["algo"] = ALGO
params["feature"] = FEATURE
params["seq_len"] = SEQ_LEN
params["seq_per_batch"] = SEQ_PER_BATCH
params["strides"] = N_STRIDES
params["epochs"] = EPOCHS
params["train_rows"] = df_train.shape[0]
params["val_rows"] = df_val.shape[0]
params["test_rows"] = df_test.shape[0]
params["norm_value"] = ref_norm
params["norm_type"] = NORM_TYPE
params["learning_rate"] = LEARNING_RATE
params["loss"] = 'adam'
params["clipvalue"] = CLIP_VALUE

In [ ]:
print(params)

{'algo': 'seq2seq', 'feature': 'ft_kettle', 'seq_len': 32, 'seq_per_batch': 1024, 'strides': 1, 'epochs': 40, 'train_rows': 521300, 'val_rows': 316800, 'test_rows': 190700, 'norm_value': 325.09125, 'norm_type': 'mean', 'learning_rate': 0.001, 'loss': 'adam', 'clipvalue': 10.0}


In [ ]:
results = analysis.classification_results(y_test, y_pred, on_threshold = 100)

metrics = {}
metrics["mae"] = analysis.mse_loss(y_test, y_pred)
metrics["mse"] = analysis.mae_loss(y_test, y_pred)
metrics["f1"] = analysis.f1(results)
metrics["accuracy"] = analysis.accuracy(results)
metrics["precision"] = analysis.precision(results)
metrics["recall"] = analysis.recall(results)

In [ ]:
print(metrics)

{'mae': 4078.35619694777, 'mse': 3.5742928677042927, 'f1': 0.8027268853855987, 'accuracy': 0.9975716953028301, 'precision': 0.6896046852122987, 'recall': 0.9602446483180428}


In [ ]:
if not SKIP_TRAINING:
    losses_to_file(1, params, history.history)

AttributeError: ignored

In [ ]:
metrics_to_file(1, params, metrics)